In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from collections import defaultdict
from scipy.stats import linregress
from pprint import pprint
import pandas as pd
import numpy as np
import gensim
import re

Я взялся решать определение тональности и постарался придумать, как это может быть полезно Теле2. Данные я разметил сам полуавтоматическим методом (кластеризовал и раскидывал руками по классам). Я разметил три класса, но в задаче позитивный и нейтральный были слиты в один класс.

### 1. Определение тональности

Обучим классификатор тональности на размеченных за ночь данных.

In [2]:
positive = open('sentiment/positive_gold.txt').read().splitlines()
negative = open('sentiment/negative_gold.txt').read().splitlines()
neutral = open('sentiment/neutral_gold.txt').read().splitlines()

In [3]:
print('Количество положительных примеров: ', len(positive))
print('Количество негативных примеров:    ', len(negative))
print('Количество нейтральных примеров:   ', len(neutral))

Количество положительных примеров:  872
Количество негативных примеров:     2646
Количество нейтральных примеров:    1007


In [4]:
train_data = positive + neutral + negative

Сделаем минимальную предобработку: уберем тэги, лишние символы на концах предложений и зацензуренные символы.

In [5]:
def preprocess(text):
    text = re.sub('  +', ' ', 
                  re.sub("<.*?>", " ", 
                      re.sub("<.*?>", " ", 
                         re.sub('[\'\" ]+\n', '\n', 
                                re.sub("\[?xx+\]?", ' ', 
                                       text)))))
    return text

In [6]:
train_data = np.array([preprocess(text) for text in train_data])

In [7]:
train_data[:10]

array(['Нет, спасибо. Подумаю',
       'Большое спасибо. Чем отличается мой тариф от этого ""Мой онлайн?',
       ' , 6 1 6 заранее огромное спасибо, а за комплимент большое спасибо',
       'дада, спасибо, вопросов нет', 'Нет, больше вопросов. Спасибо.',
       'Большое спасибо за быстрый ответ! Удачи и успехов!!!',
       'Нет, большое спасибо)',
       'В данный момент я нахожусь по адресу Томск, ул 21/5, этаж 5 в пятиэтажке, окна квартиры выходят на дом ул 122/1. Качество голосовой связи отличное, проблема только с интернетом. Спасибо за помощь.',
       'Венрно, спасибо за помощь.',
       'Здравствуйте, вчера после обеда интернет заработал. Сейчас все нормально. Спасибо.'],
      dtype='<U2701')

In [8]:
# дальше понадобится определять только негативный класс, поэтому запишем позитивные и нейтральные в одну категорию
target = [0 for i in range(len(positive))] + [0 for i in range(len(neutral))] + [1 for i in range(len(negative))]

In [9]:
target = np.array(target)

Для простоты обучим CountVectorizer на символьных нграммах.

In [10]:
vectorizer = CountVectorizer(analyzer='char', ngram_range=(2,5), max_features=10000)

In [11]:
X = vectorizer.fit_transform(train_data)

Оценим качество на кросс-валидации. Так как классы не сбалансированы, будем использовать StratifiedKFold.

Так как классификация бинарная ограничимся accuracy.

In [12]:
scores = []


kf = StratifiedKFold(n_splits=10, random_state=23)

clf = LogisticRegression(C=0.001, random_state=23)
for i, index in enumerate(kf.split(X, target)):
    print('Doing fold ', i)
    train, valid = index
    
    X_train, X_valid = X[train], X[valid]
    y_train, y_valid = target[train], target[valid]
    
    clf.fit(X_train, y_train)
    pred = clf.predict(X_valid)

    score = f1_score(pred, y_valid)
    print('Score on fold', score)
    scores.append(score)


print('Total score - ', np.mean(scores))

Doing fold  0
Score on fold 0.9555555555555556
Doing fold  1
Score on fold 0.969811320754717
Doing fold  2
Score on fold 0.9812734082397004
Doing fold  3
Score on fold 0.9589552238805971
Doing fold  4
Score on fold 0.9700374531835205
Doing fold  5
Score on fold 0.9613259668508288
Doing fold  6
Score on fold 0.9681050656660413
Doing fold  7
Score on fold 0.9625468164794008
Doing fold  8
Score on fold 0.9511278195488723
Doing fold  9
Score on fold 0.9758812615955473
Total score -  0.9654619891754782


Качество на валидации вроде бы хорошее, но датасет совсем не идеальный. Посмотрим ещё на ошибки классификатора.

In [13]:
clf = LogisticRegression(C=0.001, random_state=23)
clf.fit(X, target)

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=23, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
preds = clf.predict(X)

In [15]:
list(zip(target[preds!=target], train_data[preds!=target]))[:10]

[(0,
  'как минимум непрофессионально и некрасиво не идти на встречу постоянному клиенту и не решать возникшую проблему. если вы зависите от других операторов, то это попросту вид монополии, что очень подрывает именно вашу (так как на других я просто не рассматривал до этого момента)репутацию отличного оператора'),
 (0, 'Благодарю, всё отлично!!!!'),
 (0, 'Документы все были отправлены мною!!!'),
 (0, ' , ну круто че, теле2 топовый оператор)'),
 (0,
  'Добрый день. Что с вашим интернетом? 2 месяца пользовалась, было все отлично! В этот же месяц просто ужас какой то... в вк постоянно показывает что нет интернета, в игре так же. Я сейчас зашла в игру с билайна(там трафик кончился) так я хоть поиграла'),
 (0, 'Получилось!!! Спасибо огромное!!!)))'),
 (0,
  'Вот если честно, граждане из Теле2, вы уже достали своими скрытыми маневрами, за все годы пользованием БВК, а это с г, проблем вообще не было, а как только появились Вы, то повышение абонентской платы (на этом же номере), то изменение 

Что-то и правда не определяется как надо, а что-то явно моя ошибка при разметке.

### 2. Применение на практике

Допустим, что качество классификатор хоть и не идеальное, но тем не менее отражает реальную тональность.

Прикрутим обученный классификатор к задаче определения успешности общения с оператором тех. поддержки. Логично предположить, что Теле2 заинтересованы в том, чтобы негативность 1) как минимум не повышалась в ходе диалога и 2) как максимум уменьшалась под конец. 

Достанем из файла dialog_1.csv цепочки сообщений пользователя и припишем каждой из них тональность. Чтобы измерить динамику изменения негативности впихнем в каждую последовательность регрессию и возьмем slope полученной прямой.

In [16]:
dialog = pd.read_csv('DATA/dialogs_1.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
ids = dialog.ID.dropna().unique()

In [18]:
# Достанем ещё и отдельно ответы операторов и полные последовательности диалогов
# тут все очень не эффективно из-за того, что я торопился
sequences = []
sequences_reply = []
sequences_full = []

for idx in ids[:1000]:
    
    data = dialog[dialog.ID == idx]
    contents = data[(data['Статус'] == 'Инцидент') | (data['Статус'] == 'Новый')]['Контент'].dropna()
    contents = list(contents.apply(preprocess).values)
    sequences.append(contents)
    
    contents_reply = data[data['Статус'] == 'Ожидание ответа автора']['Контент'].dropna()
    contents_reply = list(contents_reply.apply(preprocess).values)
    sequences_reply.append(contents_reply)
    
    contents_full = data['Контент'].dropna()
    contents_full = list(contents_full.apply(preprocess).values)
    sequences_full.append(contents_full)

In [19]:
# векторизуем последовательности
# предсказываем вероятность негативного класса
# если в последовательности меньше 3 реплик, ставим нули и идем дальше
# строим регрессию, берем slope
# сохраняем вероятности, чтобы потом их легче было посмотреть

slopes = []
probability_sequences = []


for sequence in sequences:
    if len(sequence) < 3:
        slopes.append(0)
        preds = [0 for r in sequence]
        probability_sequences.append(preds)
        continue
    
    vecs = vectorizer.transform(sequence)
    preds = clf.predict_proba(vecs)
    
    preds = [x[0] for x in preds]
    probability_sequences.append(preds)
    
    slopes.append(linregress(list(range(len(sequence))), preds)[0])

Посмотрим, какие диалоги определяются как "скатившиеся".

In [20]:
sorted([(i,x) for i,x in enumerate(slopes) if x != 0], key=lambda x: x[1])[:10]

[(133, -0.19583663599528306),
 (564, -0.1940239463477752),
 (376, -0.17629244975801905),
 (365, -0.14429035168041443),
 (721, -0.1372134138989614),
 (217, -0.13181914679381873),
 (218, -0.12507367009307907),
 (358, -0.12487260201464745),
 (623, -0.12393864346379435),
 (107, -0.12051254734829878)]

In [21]:
for text, proba in zip(sequences[133], probability_sequences[133]):
    print(re.sub('  +', '', text), proba)
    print('-'*50)

'Добрый день,большая просьба к группе, помогите,пропал сын,номер сына ,полная информация у меня на стене,Александр,помогите 0.750223672179207
--------------------------------------------------
'Одного из оператора я могу узнать информацию звонков,есть свои люди,надо анонимно под запись 0.40710217591274367
--------------------------------------------------
'А я вам того же, желаю,он повешался,вы не люди 0.3585504001886409
--------------------------------------------------


Посмотрим, что в этой ситуации сказал оператор.

In [22]:
for text in sequences_full[133]:
    print(re.sub('  +', '', text))
    print('-'*50)

'Добрый день,большая просьба к группе, помогите,пропал сын,номер сына ,полная информация у меня на стене,Александр,помогите
--------------------------------------------------
'Здравствуйте, Александр! Сочувствуем вам и входим в положение. Однако детализацию звонков вы можете заказать бесплатно в личном кабинете https://tele2.ru/lk в разделе «Расходы», «Подробнее о расходах», нажав на ссылку «Заказать детализацию». Либо владелец номера с паспортом может обратиться в офис нашей компании. Стоимость заказа в офисе 10 рублей за календарный месяц. Адреса офисов и графики их работы по ссылке https://tele2.ru/offices . Рекомендуем вам незамедлительно обратиться в полицию. Мы верим, что ситуация разрешится, пожалуйста, не переживайте. Может быть подскажем вам ещё что-то?
--------------------------------------------------
'Одного из оператора я могу узнать информацию звонков,есть свои люди,надо анонимно под запись
--------------------------------------------------
'А я вам того же, желаю,он пове

Посмотрим на улучшившиеся диалоги.

In [24]:
sorted([(i,x) for i,x in enumerate(slopes) if x != 0], key=lambda x: -x[1])[:10]

[(504, 0.42264158844849326),
 (509, 0.3699203318967088),
 (592, 0.34501036803485624),
 (354, 0.33536596837208726),
 (277, 0.33363188349774836),
 (910, 0.3262727500968729),
 (694, 0.3261815411639365),
 (828, 0.31253867211081615),
 (77, 0.30050557529155564),
 (493, 0.2778947635043884)]

In [25]:
for text, proba in zip(sequences[509], probability_sequences[509]):
    print(re.sub('  +', '', text), proba)
    print('-'*50)

' , что это за тарифы такие интересно? за что 3 рубля в сутки? пакетов минут там нет, пакета интернета нет. или я что-то не понимаю? 0.14523299178753013
--------------------------------------------------
'Теле2, за что ввели абонентскую плату на тарифном плане "классический"? 0.49579778875027536
--------------------------------------------------
' , Да, удалось. Спасибо. Хорошо что можно отключить. 0.8850736555809477
--------------------------------------------------


In [28]:
for text in sequences_full[504]:
    print(re.sub('  +', ' ', text))
    print('-'*50)

'Здравствуйте теле2 . Я вчера в Евросети положила 120 р на счёт И до сих пор не пришло Я снова ходила туда где клала , сказали ждать . Тип у теле 2 задержки вечные И как часто это происходить будет !? Я отправляю смс , а счёт так и не пополнился Вчера звонила оператору Евросети Деньги закончились Минуты Я нервничаю
--------------------------------------------------
' , здравствуйте! Рекомендуем сравнить информацию в чеке в строке «Провайдер», «Поставщик услуг» или «Получатель». В этой строке должно быть указано Tele2 или Ростовская сотовая связь. Если указана иная информация, то для поиска и проведения Вашего платежа рекомендуем обратиться в платежную систему по номеру технической поддержки, который указан в чеке. Обращаем ваше внимание, максимальный срок поступления платежа – 24 часа с момента пополнения баланса. Если все данные в чеке верные, и по истечении 24 часов средства не поступят, чтобы мы могли помочь отыскать платеж, пожалуйста, пришлите нам в лс ваш номер Tele2, а также дат

Такие данные можно использовать для мониторинга успешности поддержки, поиска плохих и хороших диалогов, на которых можно потом обучать новых операторов.

### Попробуем посчитать какие-нибудь статистики и посмотреть как они коррелируют с коэффициентами наклона.

In [29]:
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import word_tokenize
from collections import Counter
from scipy.stats import spearmanr

Морфологию возьмем из pymoprhy.

In [30]:
morph = MorphAnalyzer()

def tokenizer(text):
    words = [word.lower() for word in word_tokenize(text) if word.isalnum()]
    return words


Для подсчета сложных слов посчитаем частотность слов на 100 тыс. текстов и возьмем топ10к как порог частотной и общеизвестной лексики.

In [31]:
words = dialog['Контент'].dropna().iloc[:100000].apply(lambda x: tokenizer(preprocess(x))).values

In [32]:
total_word_count = Counter()
for ws in words:
    total_word_count.update(ws)
top10k = set([word for word,c in total_word_count.most_common(10000)])

In [33]:
# я торопился и поэтому тут все непонятно

mean_answers_len = []
mean_answer_word_len = []
mean_len_of_word = []
percentage_of_unique_words = []
percent_of_complicated_words = []
percent_of_personal_pronouns = []
percent_of_imperatives = []


for sequence in sequences_reply:
    mean_len = np.mean([len(reply) for reply in sequence])
    mean_word_len = 0
    len_of_word = []
    words_count = 0
    complicated_word_count = 0
    imperatives = 0
    personal_pronouns = 0
    uniques = 0
    for reply in sequence:
        words = tokenizer(reply)
        uniques += len(set(words))
        words_count += len(words)
        complicated_word_count += len([word for word in words if word not in top10k])
        mean_word_len += len(words)/len(sequence)
        mean_len_of_word += [len(word) for word in words]
        
        for word in words:
            p = morph.parse(word)[0].tag
            if 'impr' in p:
                imperatives += 1
            elif 'NPRO' in p:
                personal_pronouns += 1
    
    if words_count < 1:
        mean_answers_len.append(0)
        mean_answer_word_len.append(0)
        mean_len_of_word.append(np.mean(0))
        percentage_of_unique_words.append(0)
        percent_of_complicated_words.append(0)
        percent_of_personal_pronouns.append(0)       
        percent_of_imperatives.append(0)
        continue
    
    mean_answers_len.append(mean_len)
    mean_answer_word_len.append(mean_word_len)
    mean_len_of_word.append(np.mean(len_of_word))
    percentage_of_unique_words.append(uniques/words_count)
    percent_of_complicated_words.append(complicated_word_count/words_count)
    percent_of_personal_pronouns.append(personal_pronouns/words_count)       
    percent_of_imperatives.append(imperatives/words_count)
        
    

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [34]:
mean_answers_len = np.array(mean_answers_len)
mean_answer_word_len = np.array(mean_answer_word_len)
mean_len_of_word = np.array(mean_len_of_word)
percentage_of_unique_words = np.array(percentage_of_unique_words)
percent_of_complicated_words = np.array(percent_of_complicated_words)
percent_of_personal_pronouns = np.array(percent_of_personal_pronouns)
percent_of_imperatives = np.array(percent_of_imperatives)

In [35]:
len(mean_answers_len)

1000

In [36]:
# оставим только примеры с ненулевым коэффициентом
good = [i for i,x in enumerate(slopes) if x!=0]

In [37]:
slopes = np.array(slopes)

In [38]:
print('Корреляция со средней длинной ответа в символах\n', spearmanr(mean_answers_len[good], slopes[good]))
print('Корреляция со средней длинной ответ в сдовах\n', spearmanr(mean_answer_word_len[good], slopes[good]))
print('Корреляция со средней длинной слова в ответе\n', spearmanr(mean_len_of_word[good], slopes[good]))
print('Корреляция с процентом "сложных" слов\n', spearmanr(percent_of_complicated_words[good], slopes[good]))
print('Корреляция с процентом местоимений\n', spearmanr(percent_of_personal_pronouns[good], slopes[good]))
print('Корреляция с процентом глаголов в императиве\n', spearmanr(percent_of_imperatives[good], slopes[good]))

Корреляция со средней длинной ответа в символах
 SpearmanrResult(correlation=0.132932009901971, pvalue=0.008079819301497168)
Корреляция со средней длинной ответ в сдовах
 SpearmanrResult(correlation=0.14311989402000183, pvalue=0.004320628081334912)
Корреляция со средней длинной слова в ответе
 SpearmanrResult(correlation=nan, pvalue=nan)
Корреляция с процентом "сложных" слов
 SpearmanrResult(correlation=0.015284636485162971, pvalue=0.7617247687678227)
Корреляция с процентом местоимений
 SpearmanrResult(correlation=0.008352780210654021, pvalue=0.8683964493223814)
Корреляция с процентом глаголов в императиве
 SpearmanrResult(correlation=0.15968161852650253, pvalue=0.0014323124214321381)


#### Возможно, это означает, что длинна ответов и процент глаголов в повелительном наклонении сказывается на удовлетворенности.